## MQB7046 MODELLING PUBLIC HEALTH DATA - Ordinal logistic regression
Prepared by Claire Choo (15/4/2024)

Ordinal Logistic Regression is a statistical model used when the dependent variable is ordinal in nature, that is it has a clear ordering or ranking, but the differences between the categories may not be equal. 
The dependent variable must be ordinal, such as a rating scale (e.g. 1 = poor, 2 = fair, 3 = good, 4 = excellent). 
The model assumes that the relationship between the predictor variables and the logit of the cumulative probabilities is the same for all logits (the proportional odds assumption). 

#### Practical 5

A study examined factors that influence the decision of whether to apply to graduate programme.  Undergraduates were asked if they are unlikely, somewhat likely, or very likely to apply to graduate programme. The outcome variable has three categories coded as 0,1,2. Data on parental educational status (0 is no tertiary education, 1 is with tertiary education), whether the undergraduate institution is public (1) or private (0), and current GPA was collected. 

In [1]:
import pandas as pd

# Load the dataset into a DataFrame
grad = pd.read_csv("open.csv")

# Display the first few rows of the DataFrame to verify that the data is loaded correctly
print(grad.head().to_markdown(tablefmt = "pretty", index = False))

+----+------+-------+--------------------+-------------+
| id | Sex  |  Age  |      Marital       | Employment  |
+----+------+-------+--------------------+-------------+
| 1  | Male | 55-64 | Married/Cohabiting |  Employed   |
| 2  | Male | 25-44 | Married/Cohabiting | Nonemployed |
| 3  | Male | 25-44 | Married/Cohabiting |  Employed   |
| 4  | Male | 55-64 | Married/Cohabiting |  Employed   |
| 5  | Male | 55-64 | Married/Cohabiting |  Employed   |
+----+------+-------+--------------------+-------------+


In [2]:
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import os
import dotenv

dotenv.load_dotenv()

llm = OpenAI()

df = SmartDataframe(grad, config = {'llm': llm})

In [3]:
df.chat("Perform a descriptive statistics analysis")

Traceback (most recent call last):
  File "/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 283, in run
    output = (self.code_generation_pipeline | self.code_execution_pipeline).run(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/pandasai/pipelines/chat/code_generator.py", line 33, in execute
    code = pipeline_context.config.llm.generate_code(input, pipeline_context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wh0102/Downloads/github/.

"Unfortunately, I was not able to answer your question, because of the following error:\n\nError code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}\n"

In [ ]:
# Perform the descriptive statistics 
# Conduct hypothesis testing

In [ ]:
# Generate odds ratio (95%CI)


In [ ]:
# run ordinal logistic regression with multiple predictors ('pared', 'public', 'gpa')


In [ ]:
# Generate predicted probabilities for each observation



In [ ]:
# Calculate cumulative probabilities


In [ ]:

# Calculate the likelihood ratio test 

In [ ]:
# Perform score test 